In [2]:
import pandas as pd
import glob

import os
from skimage import io, measure, morphology, color, filters, transform
import numpy as np

from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import tqdm


import porespy as ps

from statsmodels.tsa.api import SimpleExpSmoothing
from scipy.signal import find_peaks
import more_itertools as mit

import urllib
import torch

In [ ]:
xshift=-20
yshift=10

boundary=5

df=pd.read_csv('grades.csv')

recs={'msk22':[[780,2610],[36, 1838]],
      'msk23':[[763,2621],[44, 1824]],
      'nov23':[[755,2590],[41, 1834]],}

width_mean=[]
height_mean=[]

width_std=[]
height_std=[]

for i in tqdm(range(len(df))):
    id=df.iloc[i]['id']
    type=df.iloc[i]['type']
    rec=recs[type]
    img=io.imread(f'data/{type}/blanks_aligned_jpg/{id}.jpg')[rec[0][0]:rec[0][1],rec[1][0]:rec[1][1]]

    (C, M, Y, K) = rgb2smyk(img)

    thresh=filters.threshold_otsu(Y)
    binaryImage=Y>thresh*0.3

    lines_max, _=lines_location(binaryImage)

    img=binaryImage[lines_max[k]-xshift:lines_max[k+1]+yshift,:]

    img_tmp=np.zeros((img.shape[0]+boundary*2,img.shape[1]+boundary*2),dtype=bool)
    img_tmp[boundary:boundary+img.shape[0],boundary:boundary+img.shape[1]]=img
    words_indices,_=locate_words(img_tmp,N=3)

## Height/width of the text (in progress)

In [2]:
class Detector:
    def __init__(self, yolo_path, model_path):
        self._device = torch.device('cuda')
        self.model = torch.hub.load(yolo_path, 'custom', path=model_path, source='local')
        self.device = self._device

    def run(self, path_to_image):
        results = self.model([path_to_image])
        boxes = []
        data = results.pandas().xyxy[0]
        for i in range(data.shape[0]):
            boxes.append([data['xmin'][i], data['ymin'][i], data['xmax'][i], data['ymax'][i]])
        return boxes

class Reader:
    def __init__(
            self,
            yolo_path='D:/PROJECTS/python/aist_2023/doc2text/yolov5',
            detector_weights=None,
    ):


        if detector_weights is None:
            if not os.path.exists('D:/PROJECTS/python/aist_2023/yolo5_weights.pt'):
                urllib.request.urlretrieve(
                    "https://github.com/konverner/shiftlab_ocr/raw/main/doc2text/weights/weights.pt",
                    'D:/PROJECTS/python/aist_2023/yolo5_weights.pt')
            detector_weights = 'D:/PROJECTS/python/aist_2023/yolo5_weights.pt'

        self.detector = Detector(yolo_path, detector_weights)

    def doc2text(self, image_path):

        boxes = self.detector.run(image_path)

        return boxes


reader = Reader()

In [40]:
%%time

# boxes=reader.doc2text('62263_1.jpg')
boxes=reader.doc2text(img)
width=[]
height=[]
for box in boxes:
    width.append(box[2]-box[0])
    height.append(box[3]-box[1])

boxes=np.array(boxes)

CPU times: total: 109 ms
Wall time: 124 ms


In [42]:
width=boxes[:,2]-boxes[:,0]
width.shape

(166,)

In [ ]:
df=pd.read_csv('grades.csv')

recs={'msk22':[[780,2610],[36, 1838]],
      'msk23':[[763,2621],[44, 1824]],
      'nov23':[[755,2590],[41, 1834]],}

width_mean=[]
height_mean=[]

width_std=[]
height_std=[]

for i in tqdm(range(len(df))):
    id=df.iloc[i]['id']
    type=df.iloc[i]['type']
    rec=recs[type]
    img=io.imread(f'data/{type}/blanks_aligned_jpg/{id}.jpg')[rec[0][0]:rec[0][1],rec[1][0]:rec[1][1]]
    boxes=reader.doc2text(img)
    boxes=np.array(boxes)
    if len(boxes)>0:
        width=boxes[:,2]-boxes[:,0]
        height=boxes[:,3]-boxes[:,1]

        width_mean.append(np.mean(width))
        height_mean.append(np.mean(height))

        width_std.append(np.std(width))
        height_std.append(np.std(height))
    else:
        width_mean.append(0)
        height_mean.append(0)

        width_std.append(0)
        height_std.append(0)



  0%|          | 0/21308 [00:00<?, ?it/s]

In [68]:
df['width_mean']=width_mean
df['height_mean']=height_mean
df['width_std']=width_std
df['height_std']=height_std

In [69]:
df.to_csv('grades_wh.csv',index=False)

In [49]:
# np.save('width_mean.npy',width_mean)
# np.save('height_mean.npy',height_mean)
# np.save('width_std.npy',width_std)
# np.save('height_std.npy',height_std)

## Combine all features

In [4]:
df=pd.read_csv('grades_wh.csv')

df

,id,surname,name,code,orthography,punctuation,grade,aligned_doc,type,surname_doc,name_doc,code_doc,width_mean,height_mean,width_std,height_std
0,66717_1,харитонова,анастасия,чернослив,0.0,1.0,5,True,msk23,True,True,True,257.209327,50.055793,118.190194,8.883130
1,66717_2,рачкова,екатерина,анна,0.0,2.0,4,True,msk23,True,True,True,230.462712,53.179430,113.365492,7.599010
2,66717_3,ефимова,елена,превосходно,4.0,5.0,2,True,msk23,True,True,True,236.823072,53.649396,113.461925,9.138695
3,66717_4,деянова,юлия,асфальт,3.0,4.0,3,True,msk23,False,True,True,233.258644,53.002609,117.492249,7.919873
4,66717_5,соловьева,нелля,счастье,1.0,3.0,4,True,msk23,True,True,True,254.317524,54.462915,122.593031,8.481969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21303,00437_83,власенко,ольга,международка,0.0,2.0,4,True,nov23,True,True,True,187.962923,50.617324,90.536189,8.604293
21304,00437_84,ваймер,анна,слово,18.0,5.0,2,True,nov23,True,True,True,182.610628,47.529574,88.886865,8.781521
21305,00437_85,бердашевский,глеб,телец,11.0,12.0,2,True,nov23,True,True,True,164.288591,46.237068,74.132875,7.519557
21306,00437_86,алексеева,ирина,перфект,1.0,6.0,3,True,nov23,True,True,True,179.705911,41.768002,67.690141,6.019624
